# Using hyperparameter optimization framework Optuna

### Importing required libraries

In [ ]:
#Importing all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import missingno as msno
from prettytable import PrettyTable
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle
import os
import joblib
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer, PowerTransformer, MaxAbsScaler
from sklearn.metrics import mean_absolute_error, median_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
!pip install optuna

In [ ]:
import optuna

### Loading Dataset

In [ ]:
path = '/content/drive/MyDrive/project_asteroid/models'
X_train_imp_df = pd.read_csv(os.path.join(path, 'X_train_imp_df'))
X_val_imp_df = pd.read_csv(os.path.join(path, 'X_val_imp_df'))
X_test_imp_df = pd.read_csv(os.path.join(path, 'X_test_imp_df'))

In [ ]:
y_train = pd.read_csv('/content/drive/MyDrive/project_asteroid/model_prelim/y_train', squeeze=True)
y_val = pd.read_csv('/content/drive/MyDrive/project_asteroid/model_prelim/y_val', squeeze=True)
y_test = pd.read_csv('/content/drive/MyDrive/project_asteroid/model_prelim/y_test', squeeze=True)

##DecisionTree

In [ ]:
%%time
#  'max_depth': [5, 10, 20, 30, 40, 50],
#     'min_samples_split': [3, 5, 10, 20],
#     'min_samples_leaf': [1, 5, 10, 20, 50, 100]

def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_leaf= trial.suggest_int('min_samples_leaf', 1,100)
    min_samples_split = trial.suggest_int('min_samples_split', 3, 50)

    #defining model
    dt = DecisionTreeRegressor(min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,
                           max_depth=max_depth)

    score = np.mean(cross_val_score(dt, X_val_imp_df, y_val, cv=10, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="dt-experiment")  # Create a new study.
study.optimize(objective, n_trials=20)  # Invoke optimization of the objective function.


[I 2022-07-26 22:57:00,887] A new study created in memory with name: dt-experiment
[I 2022-07-26 22:57:05,037] Trial 0 finished with value: 0.7317170341990586 and parameters: {'max_depth': 39, 'min_samples_leaf': 86, 'min_samples_split': 23}. Best is trial 0 with value: 0.7317170341990586.
[I 2022-07-26 22:57:07,569] Trial 1 finished with value: 0.729184867190565 and parameters: {'max_depth': 14, 'min_samples_leaf': 89, 'min_samples_split': 14}. Best is trial 0 with value: 0.7317170341990586.
[I 2022-07-26 22:57:09,394] Trial 2 finished with value: 0.7741834707737042 and parameters: {'max_depth': 7, 'min_samples_leaf': 44, 'min_samples_split': 40}. Best is trial 2 with value: 0.7741834707737042.
[I 2022-07-26 22:57:12,227] Trial 3 finished with value: 0.7929506917492098 and parameters: {'max_depth': 23, 'min_samples_leaf': 35, 'min_samples_split': 33}. Best is trial 3 with value: 0.7929506917492098.
[I 2022-07-26 22:57:15,521] Trial 4 finished with value: 0.8087459601068007 and paramet

CPU times: user 2.26 s, sys: 353 ms, total: 2.62 s
Wall time: 1min 1s


##RandomForest

Trial 1

In [ ]:
# Define an objective function to be minimized.
def objective(trial):

    # Invoke suggest methods of a Trial object to generate hyperparameters.
    rf_max_depth = trial.suggest_int('rf_max_depth', 2, 32)

    regressor_obj = RandomForestRegressor(max_depth=rf_max_depth)


    regressor_obj.fit(X_train_imp_df, y_train)
    y_pred = regressor_obj.predict(X_val_imp_df)
    score = r2_score(y_val, y_pred)
    
    return score  # An objective value linked with the Trial object.


In [ ]:
study = optuna.create_study(direction='maximize', study_name="rf-experiment")  # Create a new study.
study.optimize(objective, n_trials=10)  # Invoke optimization of the objective function.

[I 2022-07-26 08:15:10,814] A new study created in memory with name: no-name-01cc15ff-f2e2-4956-ad72-a6aac1795763
[I 2022-07-26 08:16:45,909] Trial 0 finished with value: 0.8882696504517045 and parameters: {'rf_max_depth': 18}. Best is trial 0 with value: 0.8882696504517045.
[I 2022-07-26 08:18:23,191] Trial 1 finished with value: 0.8945818406884354 and parameters: {'rf_max_depth': 21}. Best is trial 1 with value: 0.8945818406884354.
[I 2022-07-26 08:20:11,642] Trial 2 finished with value: 0.8882670020033625 and parameters: {'rf_max_depth': 30}. Best is trial 1 with value: 0.8945818406884354.
[I 2022-07-26 08:21:57,006] Trial 3 finished with value: 0.8960580630775697 and parameters: {'rf_max_depth': 22}. Best is trial 3 with value: 0.8960580630775697.
[I 2022-07-26 08:22:36,844] Trial 4 finished with value: 0.8977371055024462 and parameters: {'rf_max_depth': 7}. Best is trial 4 with value: 0.8977371055024462.
[I 2022-07-26 08:24:29,901] Trial 5 finished with value: 0.9022949959885174 a

In [ ]:
trial=study_rf.best_trial
trial.params

{'rf_max_depth': 30}

Trial 2


In [ ]:
%%time
import optuna

# Define an objective function to be minimized.
def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    max_depth = trial.suggest_int('max_depth', 20, 70)
    min_samples_leaf= trial.suggest_int('min_samples_leaf', 1,100)
    n_estimators = trial.suggest_int('n_estimators', 100, 800)

    #defining model
    rf = RandomForestRegressor(n_estimators=n_estimators, 
                               max_depth=max_depth,
                               min_samples_leaf=min_samples_leaf)

   
    score = np.mean(cross_val_score(rf, X_train_imp_df, y_train, cv=5, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="rf-experiment")  # Create a new study.
study.optimize(objective, n_trials=10)  # Invoke optimization of the objective function.

[I 2022-07-26 11:21:53,083] A new study created in memory with name: rf-experiment
[I 2022-07-26 11:39:15,404] Trial 0 finished with value: 0.8507489235769992 and parameters: {'max_depth': 52, 'min_samples_leaf': 60, 'n_estimators': 387}. Best is trial 0 with value: 0.8507489235769992.
[I 2022-07-26 12:06:43,929] Trial 1 finished with value: 0.8487998720906589 and parameters: {'max_depth': 26, 'min_samples_leaf': 63, 'n_estimators': 616}. Best is trial 0 with value: 0.8507489235769992.
[I 2022-07-26 12:22:13,056] Trial 2 finished with value: 0.892840625746582 and parameters: {'max_depth': 69, 'min_samples_leaf': 14, 'n_estimators': 288}. Best is trial 2 with value: 0.892840625746582.
[I 2022-07-26 12:34:50,773] Trial 3 finished with value: 0.8605638795327042 and parameters: {'max_depth': 47, 'min_samples_leaf': 46, 'n_estimators': 270}. Best is trial 2 with value: 0.892840625746582.
[I 2022-07-26 13:04:33,413] Trial 4 finished with value: 0.8647684548353389 and parameters: {'max_depth'

CPU times: user 57.8 s, sys: 7.56 s, total: 1min 5s
Wall time: 3h 8min 41s


Trial 3

In [ ]:
%%time

# Define an objective function to be minimized.
def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    max_depth = trial.suggest_int('max_depth', 20, 70)
    min_samples_leaf= trial.suggest_int('min_samples_leaf', 1,100)
    n_estimators = trial.suggest_int('n_estimators', 100, 800)

    #defining model
    rf = RandomForestRegressor(n_estimators=n_estimators, 
                               max_depth=max_depth,
                               min_samples_leaf=min_samples_leaf)

    #fitting on validation data
    rf.fit(X_train_imp_df, y_train)
    
    y_pred = rf.predict(X_val_imp_df)
    score = r2_score(y_val, y_pred)
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="rf-experiment")  # Create a new study.
study.optimize(objective, n_trials=10)  # Invoke optimization of the objective function.

[I 2022-07-26 11:01:54,310] A new study created in memory with name: rf-experiment
[I 2022-07-26 11:07:28,017] Trial 0 finished with value: 0.7781648125337604 and parameters: {'max_depth': 50, 'min_samples_leaf': 19, 'n_estimators': 420}. Best is trial 0 with value: 0.7781648125337604.
[I 2022-07-26 11:12:29,262] Trial 1 finished with value: 0.7399966434522016 and parameters: {'max_depth': 34, 'min_samples_leaf': 47, 'n_estimators': 475}. Best is trial 0 with value: 0.7781648125337604.
[I 2022-07-26 11:16:00,208] Trial 2 finished with value: 0.7008221501892138 and parameters: {'max_depth': 31, 'min_samples_leaf': 75, 'n_estimators': 350}. Best is trial 0 with value: 0.7781648125337604.
[I 2022-07-26 11:23:55,988] Trial 3 finished with value: 0.7419264171866773 and parameters: {'max_depth': 29, 'min_samples_leaf': 45, 'n_estimators': 730}. Best is trial 0 with value: 0.7781648125337604.
[I 2022-07-26 11:29:09,081] Trial 4 finished with value: 0.6949351908195462 and parameters: {'max_dep

CPU times: user 55min 46s, sys: 3.36 s, total: 55min 49s
Wall time: 55min 50s


In [ ]:
trial=study.best_trial
trial.params

{'max_depth': 42, 'min_samples_leaf': 24, 'n_estimators': 713}

Trial 4

In [ ]:
%%time


def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    max_depth = trial.suggest_int('max_depth', 20, 100)
    min_samples_leaf= trial.suggest_int('min_samples_leaf', 1,100)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)

    #defining model
    rf = RandomForestRegressor(n_estimators=n_estimators,min_samples_leaf=min_samples_leaf,
                           max_depth=max_depth)

    score = np.mean(cross_val_score(rf, X_val_imp_df, y_val, cv=10, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="rf-experiment")  # Create a new study.
study.optimize(objective, n_trials=20)  # Invoke optimization of the objective function.


[I 2022-07-26 19:39:49,631] A new study created in memory with name: rf-experiment
[I 2022-07-26 19:42:56,140] Trial 0 finished with value: 0.7810175682316082 and parameters: {'max_depth': 65, 'min_samples_leaf': 72, 'n_estimators': 124}. Best is trial 0 with value: 0.7810175682316082.
[I 2022-07-26 20:07:13,575] Trial 1 finished with value: 0.8604820604091563 and parameters: {'max_depth': 71, 'min_samples_leaf': 27, 'n_estimators': 859}. Best is trial 1 with value: 0.8604820604091563.
[I 2022-07-26 20:21:17,685] Trial 2 finished with value: 0.7711257882711049 and parameters: {'max_depth': 33, 'min_samples_leaf': 83, 'n_estimators': 593}. Best is trial 1 with value: 0.8604820604091563.
[I 2022-07-26 20:30:54,555] Trial 3 finished with value: 0.7715861661850909 and parameters: {'max_depth': 100, 'min_samples_leaf': 83, 'n_estimators': 407}. Best is trial 1 with value: 0.8604820604091563.
[I 2022-07-26 20:35:36,822] Trial 4 finished with value: 0.8534681341628719 and parameters: {'max_de

CPU times: user 2min 6s, sys: 16 s, total: 2min 22s
Wall time: 7h 21s


##AdaBoost

In [ ]:
%%time

# ['n_estimators':[500,1000,2000],
#              'learning_rate':[.001,0.01,.1],
#              'max_depth':[1,2,4],
#              'subsample':[.5,.75,1]}

# Define an objective function to be minimized.
def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
   
    learning_rate= trial.suggest_categorical('learning_rate', [0.0001, 0.001, 0.01, 0.1, 1.0])
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
   

    #defining model
    ada = AdaBoostRegressor(n_estimators=n_estimators, learning_rate=learning_rate)

    score = np.mean(cross_val_score(ada, X_val_imp_df, y_val, cv=10, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="ada-experiment")  # Create a new study.
study.optimize(objective, n_trials=15)  # Invoke optimization of the objective function.


[I 2022-07-26 14:11:15,715] A new study created in memory with name: ada-experiment
[I 2022-07-26 14:11:21,524] Trial 0 finished with value: 0.7689221514891396 and parameters: {'learning_rate': 0.1, 'n_estimators': 5}. Best is trial 0 with value: 0.7689221514891396.
[I 2022-07-26 14:11:24,214] Trial 1 finished with value: 0.7621708146941446 and parameters: {'learning_rate': 0.0001, 'n_estimators': 3}. Best is trial 0 with value: 0.7689221514891396.
[I 2022-07-26 14:11:33,494] Trial 2 finished with value: 0.7871863579872864 and parameters: {'learning_rate': 0.01, 'n_estimators': 11}. Best is trial 2 with value: 0.7871863579872864.
[I 2022-07-26 14:11:43,604] Trial 3 finished with value: 0.7798565525007468 and parameters: {'learning_rate': 0.001, 'n_estimators': 12}. Best is trial 2 with value: 0.7871863579872864.
[I 2022-07-26 14:11:52,989] Trial 4 finished with value: 0.7791683037328923 and parameters: {'learning_rate': 0.01, 'n_estimators': 11}. Best is trial 2 with value: 0.787186357

CPU times: user 1.78 s, sys: 275 ms, total: 2.05 s
Wall time: 2min 23s


##GBDT

Trial 1

In [ ]:
%%time
import optuna

# Define an objective function to be minimized.
def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    max_depth = trial.suggest_int('max_depth', 2, 8)
    learning_rate= trial.suggest_uniform('learning_rate', .001,1)
    n_estimators = trial.suggest_int('n_estimators', 100, 800)
    subsample = trial.suggest_uniform('subsample', 0.5, 1)

    #defining model
    gbdt = GradientBoostingRegressor(n_estimators=n_estimators, 
                               max_depth=max_depth,
                               learning_rate=learning_rate, subsample=subsample)

    score = np.mean(cross_val_score(gbdt, X_train_imp_df, y_train, cv=3, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="gbdt-experiment")  # Create a new study.
study.optimize(objective, n_trials=5)  # Invoke optimization of the objective function.


[I 2022-07-26 12:27:13,519] A new study created in memory with name: gbdt-experiment
[I 2022-07-26 12:28:05,096] Trial 0 finished with value: 0.8874897227579662 and parameters: {'max_depth': 2, 'learning_rate': 0.8951557857430654, 'n_estimators': 304, 'subsample': 0.5178227560115621}. Best is trial 0 with value: 0.8874897227579662.
[I 2022-07-26 12:36:02,906] Trial 1 finished with value: 0.8593915485651059 and parameters: {'max_depth': 8, 'learning_rate': 0.8777621496132806, 'n_estimators': 534, 'subsample': 0.782827143030584}. Best is trial 0 with value: 0.8874897227579662.
[I 2022-07-26 12:39:34,988] Trial 2 finished with value: 0.9136453662286016 and parameters: {'max_depth': 3, 'learning_rate': 0.5018882283490136, 'n_estimators': 512, 'subsample': 0.9218009602278944}. Best is trial 2 with value: 0.9136453662286016.
[I 2022-07-26 12:48:41,668] Trial 3 finished with value: 0.8962665214840952 and parameters: {'max_depth': 6, 'learning_rate': 0.7967868044255226, 'n_estimators': 721, 's

CPU times: user 7.53 s, sys: 1.13 s, total: 8.66 s
Wall time: 24min 5s


Trial 2
cross validation on validation set - 20 Trials

In [ ]:
%%time

def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    max_depth = trial.suggest_int('max_depth', 2, 8)
    learning_rate= trial.suggest_categorical('learning_rate', [.001, 0.01,.1])
    n_estimators = trial.suggest_int('n_estimators', 100, 800)
    subsample = trial.suggest_discrete_uniform('subsample',  0.1, 1.0, 0.1)

    #defining model
    gbdt = GradientBoostingRegressor(n_estimators=n_estimators, 
                               max_depth=max_depth,
                               learning_rate=learning_rate, subsample=subsample)

    score = np.mean(cross_val_score(gbdt, X_val_imp_df, y_val, cv=3, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="gbdt-experiment")  # Create a new study.
study.optimize(objective, n_trials=20)  # Invoke optimization of the objective function.


[I 2022-07-26 13:14:27,681] A new study created in memory with name: gbdt-experiment
[I 2022-07-26 13:14:58,284] Trial 0 finished with value: 0.8619234786452683 and parameters: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 293, 'subsample': 0.5}. Best is trial 0 with value: 0.8619234786452683.
[I 2022-07-26 13:15:28,444] Trial 1 finished with value: 0.8417030307178625 and parameters: {'max_depth': 6, 'learning_rate': 0.1, 'n_estimators': 254, 'subsample': 0.30000000000000004}. Best is trial 0 with value: 0.8619234786452683.
[I 2022-07-26 13:16:43,231] Trial 2 finished with value: 0.8455923202016128 and parameters: {'max_depth': 8, 'learning_rate': 0.1, 'n_estimators': 222, 'subsample': 0.7000000000000001}. Best is trial 0 with value: 0.8619234786452683.
[I 2022-07-26 13:22:00,220] Trial 3 finished with value: 0.848681533774284 and parameters: {'max_depth': 7, 'learning_rate': 0.01, 'n_estimators': 766, 'subsample': 1.0}. Best is trial 0 with value: 0.8619234786452683.
[I 2022-

CPU times: user 9.45 s, sys: 1.27 s, total: 10.7 s
Wall time: 26min 5s


Trial 3

In [ ]:
%%time

def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    max_depth = trial.suggest_int('max_depth', 2, 8)
    learning_rate= trial.suggest_categorical('learning_rate', [.001, 0.01,.1])
    n_estimators = trial.suggest_int('n_estimators', 100, 800)
    subsample = trial.suggest_discrete_uniform('subsample',  0.1, 1.0, 0.1)

    #defining model
    gbdt = GradientBoostingRegressor(n_estimators=n_estimators, 
                               max_depth=max_depth,
                               learning_rate=learning_rate, subsample=subsample)

    score = np.mean(cross_val_score(gbdt, X_val_imp_df, y_val, cv=10, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="gbdt-experiment")  # Create a new study.
study.optimize(objective, n_trials=30)  # Invoke optimization of the objective function.


[I 2022-07-26 15:00:21,003] A new study created in memory with name: gbdt-experiment
[I 2022-07-26 15:07:52,198] Trial 0 finished with value: 0.8840700833263015 and parameters: {'max_depth': 8, 'learning_rate': 0.01, 'n_estimators': 231, 'subsample': 1.0}. Best is trial 0 with value: 0.8840700833263015.
[I 2022-07-26 15:17:49,322] Trial 1 finished with value: 0.919673966798177 and parameters: {'max_depth': 7, 'learning_rate': 0.01, 'n_estimators': 488, 'subsample': 0.7000000000000001}. Best is trial 1 with value: 0.919673966798177.
[I 2022-07-26 15:21:08,644] Trial 2 finished with value: 0.6483954436181556 and parameters: {'max_depth': 5, 'learning_rate': 0.001, 'n_estimators': 694, 'subsample': 0.2}. Best is trial 1 with value: 0.919673966798177.
[I 2022-07-26 15:30:42,431] Trial 3 finished with value: 0.9208991741208085 and parameters: {'max_depth': 7, 'learning_rate': 0.01, 'n_estimators': 483, 'subsample': 0.7000000000000001}. Best is trial 3 with value: 0.9208991741208085.
[I 2022

CPU times: user 35.6 s, sys: 4.65 s, total: 40.3 s
Wall time: 1h 53min 55s


##XGBoost

In [ ]:
%%time

def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    max_depth = trial.suggest_int('max_depth', 2, 8)
    gamma= trial.suggest_categorical('gamma', [0, 0.1, 0.2])
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    reg_alpha = trial.suggest_float('reg_alpha', 0.1, 20)

    #defining model
    xg = XGBRegressor(n_estimators=n_estimators, 
                               max_depth=max_depth,
                               gamma=gamma, reg_alpha=reg_alpha)

    score = np.mean(cross_val_score(xg, X_val_imp_df, y_val, cv=10, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="xg-experiment")  # Create a new study.
study.optimize(objective, n_trials=20)  # Invoke optimization of the objective function.


[I 2022-07-26 17:16:40,846] A new study created in memory with name: xg-experiment
[I 2022-07-26 17:17:33,933] Trial 0 finished with value: 0.9260953238326568 and parameters: {'max_depth': 4, 'gamma': 0.2, 'n_estimators': 201, 'reg_alpha': 5.24796428913678}. Best is trial 0 with value: 0.9260953238326568.
[I 2022-07-26 17:21:32,126] Trial 1 finished with value: 0.922614252505595 and parameters: {'max_depth': 8, 'gamma': 0, 'n_estimators': 630, 'reg_alpha': 19.55947064409942}. Best is trial 0 with value: 0.9260953238326568.
[I 2022-07-26 17:23:26,872] Trial 2 finished with value: 0.9254252678852557 and parameters: {'max_depth': 2, 'gamma': 0.1, 'n_estimators': 799, 'reg_alpha': 11.674576087132063}. Best is trial 0 with value: 0.9260953238326568.
[I 2022-07-26 17:27:56,880] Trial 3 finished with value: 0.9240039993416904 and parameters: {'max_depth': 6, 'gamma': 0.2, 'n_estimators': 867, 'reg_alpha': 17.495942995333515}. Best is trial 0 with value: 0.9260953238326568.
[I 2022-07-26 17:31

CPU times: user 18.8 s, sys: 2 s, total: 20.8 s
Wall time: 49min 32s


##CatBoost

In [ ]:
%%time

def objective(trial):
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.3)
    colsample_bylevel = trial.suggest_float("colsample_bylevel", 0.01, 0.1)
    max_depth = trial.suggest_int("max_depth", 1, 15)
    boosting_type = trial.suggest_categorical("boosting_type", ["Ordered", "Plain"])

    #defining model
    ct = CatBoostRegressor(learning_rate=learning_rate,colsample_bylevel=colsample_bylevel,
                           max_depth=max_depth, boosting_type=boosting_type)

    score = np.mean(cross_val_score(ct, X_val_imp_df, y_val, cv=10, n_jobs=-1))
    
    return score  # An objective value linked with the Trial object.


study = optuna.create_study(direction='maximize', study_name="ct-experiment")  # Create a new study.
study.optimize(objective, n_trials=20)  # Invoke optimization of the objective function.


[I 2022-07-26 18:20:24,064] A new study created in memory with name: ct-experiment
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[I 2022-07-26 18:25:16,641] Trial 0 finished with value: 0.5003924994810199 and parameters: {'learning_rate': 0.0016169667956435194, 'colsample_bylevel': 0.07445540651544588, 'max_depth': 15, 'boosting_type': 'Plain'}. Best is trial 0 with value: 0.5003924994810199.
[I 2022-07-26 18:27:15,870] Trial 1 finished with value: 0.35216027505800973 and parameters: {'learning_rate': 0.0010781384932626163, 'colsample_bylevel': 0.060423075025228035, 'max_depth': 9, 'boosting_type': 'Ordered'}. Best is trial 0 with value: 0.5003924994810199.
[I 2022-07-26 18:29:25,319] Trial 2 finished with value: 0.8153408924423579 and parameters: {'learning_r

KeyboardInterrupt: ignored